## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [4]:
df_ps = spark.read.json("/user/sprenner/training_official_april_hourly_proportion_prefetching.json")

In [5]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0   1624        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902          197902               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0          111420          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

In [6]:
df_ps = df_ps.fillna(0,['count', 'int_percentage','max_percentage'])

In [7]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0      0        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902               0               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0               0          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

### make buckets

In [8]:
from pyspark.ml.feature import Bucketizer

splits_percentage = [-float("inf"), 16, float("inf")]
bucketizer = Bucketizer(splits=splits_percentage, inputCol="max_percentage", outputCol="bucketedPercentage")
df_ps = bucketizer.transform(df_ps)

splits_hits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits_hits, inputCol="hits", outputCol="bucketedHits")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

In [9]:
spark.sql("SELECT bucketedPercentage, count(*) \
            FROM bucket \
            GROUP BY  bucketedPercentage \
            ORDER BY 1 ASC").toPandas()

bucketedPercentage  count(1)
0                 0.0    247128
1                 1.0    530158

In [10]:
spark.sql("SELECT bucketedHits, count(*) \
            FROM bucket \
            GROUP BY  bucketedHits \
            ORDER BY 1 ASC").toPandas()

bucketedHits  count(1)
0           0.0    541829
1           1.0    235457

In [11]:
final_data = bucketedData.withColumn("prefetch_bucket", when(bucketedData["bucketedPercentage"] == 1.0, "prefetch").otherwise("noprefetch"))

In [12]:
final_data = final_data.withColumn("class_bucket", when(final_data["bucketedHits"] == 0.0, "nocache").otherwise("cache"))

In [13]:
final_data.createOrReplaceTempView("final")
spark.sql("SELECT prefetch_bucket, count(*) \
            FROM final \
            GROUP BY  prefetch_bucket \
            ORDER BY 1 ASC").toPandas()

prefetch_bucket  count(1)
0      noprefetch    247128
1        prefetch    530158

In [14]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    235457
1      nocache    541829

In [15]:
final_data.describe().toPandas()

summary  account           accountHash         clientState  \
0   count   777286                777286              777286   
1    mean     None   1.935684591732567E8                None   
2  stddev     None  1.1361317241087065E9                None   
3     min  aagaard           -2127257360  CannotAuthenticate   
4     max   zzheng            2143592175      SourceNotFound   

        clientStateHash              count  \
0                777286             777286   
1   -9.74016245191591E8  3673.703860612439   
2  2.6212986830087826E8   7419.35514431962   
3           -1922357674                  0   
4            1269532611             199996   

                                             dataset           datasetHash  \
0                                             777286                777286   
1                                               None  -1.072936835947129E7   
2                                               None   1.258675129155887E9   
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...           -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       777286                777286          777286                777286   
1         None  -7.552649492442358E8            None  1.3991617429934412E7   
2         None   8.314865501449597E8            None  1.3045708543551397E9   
3    data10_hi           -2057006107     c1130_m1847           -2146991447   
4   mc16_valid            2109319881      x557_m1947            2146405934   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   777286                777286    777286        777286       777286   
1     None   4.947060556405943E8      None    5.937808E7         None   
2     None  1.1700303292632794E9      None           0.0         None   
3      AOD           -2070622869  download      59378080       1.13.2   
4      log            2124512252  download      59378080       1.15.5   

       eventVersionHash   fileScope                            filename  \
0                777286      777286                              777286   
1  -1.992468944589716E9        None                                None   
2   3.073273964746096E8        None                                None   
3           -2044360925   data10_hi    AOD.05371635._000001.pool.root.1   
4            1261702107  mc16_valid  log.13787275._000072.job.log.tgz.1   

           filenameHash       filesize_double               hits  \
0                777286                777286             777286   
1   -248557.46481861244    6.11402028756827E8  2.010655022732945   
2  1.2402658149710774E9  1.2408771809945793E9   2.33759009262949   
3           -2147478094                 304.0                  1   
4            2147482617       1.1991446515E10                 40   

                   hostname           hostnameHash           hour  \
0                    777286                 777286         777286   
1                      None  -1.1804844919942595E8           None   
2                      None    1.228645222705116E9           None   
3    ReCaS-ui-01.cs.infn.it            -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu             2129595856  2018-04-30 23   

                hourHash                 id       id_proportion  \
0                 777286             777286              579384   
1  -2.5176970566176414E7           388643.5   370425.3149948911   
2   1.2215909376898773E9  224383.2850061846  228391.37208069928   
3            -2138243719                  1                   2   
4             2144044799             777286              777286   

       int_percentage                                     join_key_april  \
0              777286                                             777286   
1  23.303134238877323                                               None   
2  28.438942532079476                                    

In [16]:
#final_data = final_data.withColumn("fileScopeHash", hash(final_data['fileScope'])) ## maybe this was what caused randomization

## Bucketing and Hashing End

In [17]:
final_data = final_data.orderBy("traceTimeentryUnix", ascending=True)

In [18]:
train_data, test_data = final_data.randomSplit([.8,.2],seed=1234)

In [19]:
train_data.describe().toPandas()
#80: 621574

summary  account           accountHash         clientState  \
0   count   621574                621574              621574   
1    mean     None  1.9308062682711148E8                None   
2  stddev     None   1.136771193710722E9                None   
3     min  aagaard           -2127257360  CannotAuthenticate   
4     max   zzheng            2143592175      SourceNotFound   

        clientStateHash               count  \
0                621574              621574   
1   -9.73958943068111E8   3671.944508618443   
2  2.6197450511623117E8  7424.0386893172945   
3           -1922357674                   0   
4            1269532611              199996   

                                             dataset            datasetHash  \
0                                             621574                 621574   
1                                               None  -1.0783000198251536E7   
2                                               None   1.2586974128611562E9   
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...            -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...             2147407155   

  datasetScope      datasetScopeHash dataset_version  dataset_versionHash  \
0       621574                621574          621574               621574   
1         None  -7.558718609587675E8            None  1.487149474973374E7   
2         None   8.309742455688952E8            None  1.304091153717598E9   
3    data10_hi           -2057006107     c1130_m1847          -2146991447   
4   mc16_valid            2109319881      x557_m1947           2146405934   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   621574                621574    621574        621574       621574   
1     None  4.9550641754313564E8      None    5.937808E7         None   
2     None  1.1703909475333602E9      None           0.0         None   
3      AOD           -2070622869  download      59378080       1.13.2   
4      log            2124512252  download      59378080       1.15.5   

        eventVersionHash   fileScope                            filename  \
0                 621574      621574                              621574   
1  -1.9925156808486648E9        None                                None   
2    3.071925837933101E8        None                                None   
3            -2044360925   data10_hi    AOD.05371635._000001.pool.root.1   
4             1261702107  mc16_valid  log.13787275._000072.job.log.tgz.1   

          filenameHash       filesize_double                hits  \
0               621574                621574              621574   
1  -1333130.8660175619   6.114059433382622E8   2.009464681598651   
2  1.239637998599927E9  1.2425065086809037E9  2.3368697111680454   
3          -2147454916                 304.0                   1   
4           2147482617       1.1931263819E10                  40   

                   hostname          hostnameHash           hour  \
0                    621574                621574         621574   
1                      None  -1.186555335336565E8           None   
2                      None  1.2284216734054549E9           None   
3    ReCaS-ui-01.cs.infn.it           -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu            2129595856  2018-04-30 23   

                hourHash                  id       id_proportion  \
0                 621574              621574              463479   
1  -2.5605492000682138E7  388536.71933993377  370305.14979535213   
2   1.2210613447248888E9   224355.7892991933  228349.93905882424   
3            -2138243719                   1                   2   
4             2144044799              777286              777286   

       int_percentage                                     join_key_april  \
0              621574                                             621574   
1  23.322497401757474                                               None   
2  28.448680377487996                        

In [20]:
test_data.describe().toPandas()
#80: 155671

summary   account           accountHash         clientState  \
0   count    155671                155671              155671   
1    mean      None  1.9647875568721214E8                None   
2  stddev      None  1.1340800441040735E9                None   
3     min   aagaard           -2124932851  CannotAuthenticate   
4     max  zmeadows            2143592175      SourceNotFound   

        clientStateHash               count  \
0                155671              155671   
1  -9.741331701632802E8  3688.3033962652003   
2  2.6249229466651514E8   7435.598443863747   
3           -1913538301                   0   
4             634051254              179999   

                                             dataset           datasetHash  \
0                                             155671                155671   
1                                               None     -9518646.67866205   
2                                               None  1.2570328355800724E9   
3  data15_13TeV.00266904.physics_Main.deriv.DAOD_...           -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...            2145317357   

   datasetScope     datasetScopeHash dataset_version   dataset_versionHash  \
0        155671               155671          155671                155671   
1          None  -7.53486744349622E8            None  1.1521550428763226E7   
2          None  8.325918030931273E8            None  1.3066698086049094E9   
3  data15_13TeV          -2057006107     c1130_m1847           -2146330186   
4    mc16_valid           2109319881      x557_m1947            2146405934   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   155671                155671    155671        155671       155671   
1     None  4.9382989118487066E8      None    5.937808E7         None   
2     None  1.1692624945480888E9      None           0.0         None   
3      AOD           -2055687258  download      59378080       1.13.2   
4      log            2124512252  download      59378080       1.15.5   

       eventVersionHash     fileScope                            filename  \
0                155671        155671                              155671   
1  -1.992483572412421E9          None                                None   
2   3.072667688500302E8          None                                None   
3           -2044360925  data15_13TeV    AOD.05371635._000003.pool.root.1   
4            -153312916    mc16_valid  log.13787275._000071.job.log.tgz.1   

           filenameHash      filesize_double                hits  \
0                155671               155671              155671   
1     5448245.994738905   6.11014469747249E8   2.013098136454446   
2  1.2394237375992513E9  1.236874792755259E9  2.3285761592441983   
3           -2147389276                304.0                   1   
4            2147482617      1.1931263819E10                  40   

                   hostname           hostnameHash           hour  \
0                    155671                 155671         155671   
1                      None  -1.1451033333169313E8           None   
2                      None    1.229615231277225E9           None   
3  acas0003.usatlas.bnl.gov            -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu             2129595856  2018-04-30 23   

               hourHash                  id       id_proportion  \
0                155671              155671              115863   
1  -2.690245198790398E7   388877.0983869828   370781.4821470185   
2   1.222987197157006E9  224514.40953895496  228660.71408002963   
3           -2138243719                   2                   2   
4            2144044799              777285              777285   

       int_percentage                                     join_key_april  \
0              155671                                             155671   
1  23.184909199529777                                               None   
2   28.31526172062549            

In [21]:
train_data.write.format("json").save("/user/sprenner/training_april_official_hourly_prefetching_train_random15.json")
test_data.write.format("json").save("/user/sprenner/training_april_official_hourly_prefetching_test_random15.json")